## Chapter 2-6, 2강 NLP 임베딩 기반 검색 — TF‑IDF vs SBERT vs Hybrid (CPU)

- 목표: 의미/키워드 검색 방식을 비교하고, 하이브리드로 직관적 성능 차이를 체감
- 제약: CPU 전용 환경(훈련/추론 모두 CPU)
- 데이터: AG News 코퍼스(소규모 서브셋)


### 구성 (Overview)
- 0. 환경 설정 및 라이브러리
- 1. 데이터 로드 및 corpus 생성 (AG News)
- 2. 임베딩 2가지: SBERT / TF‑IDF
- 3. 단일 질문 검색: TF‑IDF / SBERT / Hybrid
- 4. 결과 비교: 직관적 성능 분석(토픽/키워드/문맥)
- 5. 추가: 하이브리드 가중치/속도/stopwords 영향


### 0. 환경 설정 및 라이브러리


In [1]:
# =========================
# 0. 환경 설정 및 라이브러리
# =========================

# 표준 라이브러리
import os, time, random
from typing import List, Tuple
from contextlib import contextmanager

# 수치/데이터/시각화
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 사이킷런
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text as sk_text

# datasets / sentence-transformers (CPU 고정)
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings("ignore", message=".*matmul.*")

# -----------------------------------------
# Matplotlib: 한글 폰트 및 마이너스 기호 설정
# -----------------------------------------
plt.rcParams["font.family"] = "AppleGothic"
plt.rcParams["axes.unicode_minus"] = False


/Users/kimjinseok/Desktop/metacode-lecture/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------------------
# 재현성(시드) 고정
# ------------------------
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)

set_seed(42)

# ------------------------
# 간단 타이머
# ------------------------
@contextmanager
def timer(msg: str):
    t0 = time.perf_counter()
    yield
    print(f"[TIME] {msg}: {time.perf_counter() - t0:.2f}s")

### 1. 데이터 로드 및 corpus 생성 (AG News)


In [3]:
# ------------------------
# AG News 서브셋 로더
# ------------------------
label_names: List[str] = ["World", "Sports", "Business", "Sci/Tech"]

def load_ag_news_subset(n_per_class: int = 200) -> Tuple[List[str], List[int]]:
    """
    AG News에서 클래스별로 동일 개수(n_per_class)만큼 샘플을 추출해
    텍스트 리스트(xs)와 정수 라벨 리스트(ys)를 반환합니다.

    Args:
        n_per_class (int): 각 클래스(0~3)에서 가져올 샘플 수. 기본 200.

    Returns:
        xs (List[str]): 뉴스 텍스트 리스트
        ys (List[int]): 라벨 리스트 (0=World, 1=Sports, 2=Business, 3=Sci/Tech)
    """
    # -------------------------
    # 1) AG News train split 로드 (총 120k 샘플)
    # -------------------------
    ds = load_dataset("ag_news", split="train")

    xs, ys = [], []

    # -------------------------
    # 2) 클래스별 균등 샘플링
    #    - 라벨(0~3) 기준으로 필터링
    #    - 앞에서 n_per_class개만 선택
    #    - 필요 시 .shuffle(seed=42)로 무작위 추출 가능
    # -------------------------
    for lab in range(4):
        sub = ds.filter(lambda ex: ex["label"] == lab)
        # sub = sub.shuffle(seed=42)  # 무작위 샘플이 필요하면 주석 해제
        sub = sub.select(range(n_per_class))

        # -------------------------
        # 3) 결과 누적: 텍스트/라벨
        # -------------------------
        xs += [r["text"] for r in sub]
        ys += [int(r["label"]) for r in sub]

    # -------------------------
    # 4) 반환
    # -------------------------
    return xs, ys

In [4]:
with timer("AG News 로드"):
    # 각 클래스에서 200개씩 불러와 텍스트/라벨 리스트 생성
    # label_names: ["World", "Sports", "Business", "Sci/Tech"]
    corpus_texts, corpus_labels = load_ag_news_subset(n_per_class=200)

# 전체 샘플 수와 라벨 이름 확인
print(len(corpus_texts), "samples")
print("label_names:", label_names)

# -------------------------
# 클래스별 샘플 예시 2개씩 출력
# - shown 딕셔너리로 클래스별 출력 개수 제한(2개)
# - 텍스트는 길수 있으므로 앞 120자만 미리보기 형태로 출력
# -------------------------
print("\n[클래스별 샘플 예시]")
shown = {i: 0 for i in range(4)}  # 각 라벨별로 몇 개를 보여줬는지 카운트
for t, lab in zip(corpus_texts, corpus_labels):
    if shown[lab] < 2:
        print(f"[{label_names[lab]}] {str(t)[:120]}...")
        shown[lab] += 1



[TIME] AG News 로드: 4.15s
800 samples
label_names: ['World', 'Sports', 'Business', 'Sci/Tech']

[클래스별 샘플 예시]
[World] Venezuelans Vote Early in Referendum on Chavez Rule (Reuters) Reuters - Venezuelans turned out early\and in large number...
[World] S.Koreans Clash with Police on Iraq Troop Dispatch (Reuters) Reuters - South Korean police used water cannon in\central ...
[Sports] Phelps, Thorpe Advance in 200 Freestyle (AP) AP - Michael Phelps took care of qualifying for the Olympic 200-meter frees...
[Sports] Reds Knock Padres Out of Wild-Card Lead (AP) AP - Wily Mo Pena homered twice and drove in four runs, helping the Cincinn...
[Business] Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics,...
[Business] Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputat...
[Sci/Tech] 'Madden,' 'ESPN' Football Score in Different Ways (Reuters) Reuters - Was absente

### 2. 임베딩 2가지: SBERT / TF‑IDF


In [5]:
# ------------------------
# TF-IDF 인덱스 구축
# ------------------------
# stopwords: 영어 불용어 사용(키워드 매칭 과다 방지)
stop_words = list(sk_text.ENGLISH_STOP_WORDS) 

In [6]:
# -------------------------
# TF-IDF 벡터화
# -------------------------
# - max_features=30000 : 전체 단어/바이그램 중 상위 3만 개만 사용(빈도 기준)
# - ngram_range=(1, 2) : 유니그램과 바이그램을 함께 사용
# - stop_words        : 불용어 리스트(예: 영어면 'english' 또는 커스텀 리스트)
vectorizer = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    stop_words=stop_words
)

with timer("TF-IDF fit_transform(corpus)"):
    X_tfidf = vectorizer.fit_transform(corpus_texts)

# 결과 차원: (문서 개수, 사용된 토큰 개수)
print("TF-IDF shape:", X_tfidf.shape)

[TIME] TF-IDF fit_transform(corpus): 0.08s
TF-IDF shape: (800, 21987)


In [7]:
# ------------------------
# SBERT 임베딩 구축 (CPU)
# ------------------------

def sbert_encode(texts: List[str], batch_size: int = 64,
                 model_name: str = "sentence-transformers/all-MiniLM-L6-v2") -> np.ndarray:
    """
    Sentence-BERT 모델로 문장 임베딩을 생성합니다. (CPU 기본)
    - encode()의 normalize_embeddings=False로 두고, 아래에서 수동 L2 정규화 수행
    - NaN/Inf 방지도 함께 처리

    Args:
        texts (List[str]): 인코딩할 문장(문서) 리스트
        batch_size (int): 배치 크기 (메모리/속도 트레이드오프)
        model_name (str): sentence-transformers 허브 모델 이름

    Returns:
        np.ndarray: (N, D) 형태의 L2-정규화된 임베딩 배열 (float64)
    """
    # 모델 로드 (CPU). GPU 사용 시 device="cuda"로 변경 가능
    model = SentenceTransformer(model_name, device="cpu")

    arrs = []
    # 배치 단위로 순회
    for i in range(0, len(texts), batch_size):
        # convert_to_numpy=True: 바로 NumPy 배열 반환
        # normalize_embeddings=False: 아래에서 직접 L2 정규화 예정
        arr = model.encode(
            texts[i:i+batch_size],
            batch_size=batch_size,
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=False
        ).astype(np.float64)

        # ----- 안정적인 L2 정규화 -----
        # 분모가 0이 되는 경우를 방지하기 위해 clip으로 하한선 설정
        norms = np.linalg.norm(arr, axis=1, keepdims=True)
        arr = arr / np.clip(norms, 1e-12, None)

        # 숫자 안정성: NaN/±Inf → 0.0 치환
        arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)

        arrs.append(arr)

    # 배치 결과를 하나의 (N, D) 배열로 결합
    return np.vstack(arrs)



In [8]:
# -------------------------
# 실행: 코퍼스 임베딩 생성
# -------------------------
with timer("SBERT encode(corpus)"):
    # sbert_encode:
    # - Sentence-BERT로 코퍼스의 각 문장을 D차원 임베딩으로 변환
    X_sbert = sbert_encode(corpus_texts)

# 임베딩 배열의 크기 출력 (예: (800, 384) → 800문서, 384차원 임베딩)
print("SBERT shape:", X_sbert.shape)

[TIME] SBERT encode(corpus): 27.86s
SBERT shape: (800, 384)


### 3. 단일 질문 검색: TF‑IDF / SBERT / Hybrid


In [9]:
# ------------------------
# 검색 함수 3종
# ------------------------

def search_tfidf(query: str, topk: int = 5):
    """
    TF-IDF 기반 키워드 매칭 검색.
    - vectorizer: 이미 fit된 TfidfVectorizer
    - X_tfidf  : (N, V) CSR 희소행렬. 일반적으로 L2 정규화(norm='l2')가 기본.
    - 코사인 유사도 기반으로 가장 유사한 문서 인덱스와 점수를 반환.

    Args:
        query (str): 질의문
        topk (int): 상위 반환 개수

    Returns:
        List[Tuple[int, float]]: (문서 인덱스, 유사도 점수) 리스트
    """
    # (1, V) sparse row
    qv = vectorizer.transform([query])
    # sparse 간 연산 지원. vectorizer가 L2 정규화했다면 linear_kernel로 대체 가능.
    sims = cosine_similarity(qv, X_tfidf)[0]  # shape: (N,)
    idx = np.argsort(-sims)[:topk]
    
    return [(int(i), float(sims[i])) for i in idx]

In [10]:
def search_sbert(query: str, topk: int = 5):
    """
    SBERT(문장 임베딩) 기반语義 검색.
    - sbert_encode: 내부에서 L2 정규화 수행 → 코사인 유사도 = 내적
    - X_sbert     : (N, D) 실수 배열, 각 행은 단위벡터.

    Args:
        query (str): 질의문
        topk (int): 상위 반환 개수

    Returns:
        List[Tuple[int, float]]: (문서 인덱스, 유사도 점수) 리스트
    """
    # (1, D) 배열
    qv = sbert_encode([query])           # 이미 L2 정규화된 단위벡터
    sims = (X_sbert @ qv.T).ravel()      # 내적 = 코사인 유사도
    idx = np.argsort(-sims)[:topk]
    
    return [(int(i), float(sims[i])) for i in idx]

In [11]:
def search_hybrid(query: str, topk: int = 5, alpha: float = 0.5):
    """
    하이브리드 검색(TF-IDF + SBERT 가중 합).
    - alpha=1.0이면 SBERT만, 0.0이면 TF-IDF만 사용.
    - 서로 다른 점수 스케일을 단순 합산하므로, alpha는 데이터/도메인에 맞게 튜닝 권장.

    Args:
        query (str): 질의문
        topk (int): 상위 반환 개수
        alpha (float): SBERT 가중치 [0, 1]

    Returns:
        List[Tuple[int, float]]: (문서 인덱스, 하이브리드 점수) 리스트
    """
    # TF-IDF 쿼리 벡터 및 유사도
    qv_tfidf = vectorizer.transform([query])
    sims_tfidf = cosine_similarity(qv_tfidf, X_tfidf)[0]

    # SBERT 쿼리 벡터 및 유사도(내적=코사인)
    qv_sbert = sbert_encode([query])
    sims_sbert = (X_sbert @ qv_sbert.T).ravel()

    # 가중 합
    sims = alpha * sims_sbert + (1 - alpha) * sims_tfidf
    idx = np.argsort(-sims)[:topk]
    
    return [(int(i), float(sims[i])) for i in idx]


In [13]:
query = "What is the impact of interest rate hikes on the stock market?"

In [14]:
# ------------------------
# 단일 고정 질의(지시 사항대로)
# ------------------------
#

# 결과 수집 리스트
rows_tfidf, rows_sbert, rows_hybrid = [], [], []

# ------------------------
# TF-IDF 검색 및 결과 수집
# ------------------------
# search_tfidf(query, topk=5) → [(문서인덱스, 유사도점수), ...]
for k, (i, s) in enumerate(search_tfidf(query, topk=5), start=1):
    print(f"- ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i]}")
    rows_tfidf.append({
        "rank": k,
        "score": float(s),
        "doc_id": int(i),
        "label": label_names[corpus_labels[i]],
        "text": corpus_texts[i]
    })

- (0.188) [Business] Veteran inventor in market float Trevor Baylis, the veteran inventor famous for creating the Freeplay clockwork radio, is planning to float his company on the stock market.
- (0.180) [Business] In a Down Market, Head Toward Value Funds There is little cause for celebration in the stock market these days, but investors in value-focused mutual funds have reason to feel a bit smug -- if only because they've lost less than the folks who stuck with growth.
- (0.172) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
- (0.172) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market this week during the depth of the\summer doldrums.
- (0.170) [Busi

In [15]:
# ------------------------
# SBERT 검색 및 결과 수집
# ------------------------
# search_sbert(query, topk=5) → [(문서인덱스, 코사인유사도), ...]
for k, (i, s) in enumerate(search_sbert(query, topk=5), start=1):
    print(f"- ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i]}")
    rows_sbert.append({
        "rank": k,
        "score": float(s),
        "doc_id": int(i),
        "label": label_names[corpus_labels[i]],
        "text": corpus_texts[i]
    })

- (0.458) [World] Election-Year Rate Hike Puzzles Some WASHINGTON - Going against conventional wisdom, the Federal Reserve is raising interest rates in an election year. And it is Fed Chairman Alan Greenspan, a Republican, who is leading the charge even though an incumbent Republican in the White House is facing voter unrest about the state of the economy...
- (0.434) [Business] South Korea lowers interest rates South Korea's central bank cuts interest rates by a quarter percentage point to 3.5 in a bid to drive growth in the economy.
- (0.396) [Business] Stocks Higher on Oil Price Relief (Reuters) Reuters - U.S. stocks gained on Monday, getting\a boost from lower oil prices after news the Venezuelan\president survived a recall eased fears about the country's oil\exports.
- (0.374) [Business] Will Schwab Reward Patience? The company saw an improvement in its trades, but will this market be kind to the brokerages?
- (0.373) [World] Stocks Higher Despite Soaring Oil Prices NEW YORK - Wal

In [16]:
# ------------------------
# Hybrid 검색 및 결과 수집
# ------------------------
# search_hybrid(query, topk=5, alpha=0.6)
# - alpha: SBERT 가중치(0~1). 1.0=SBERT만, 0.0=TF-IDF만
for k, (i, s) in enumerate(search_hybrid(query, topk=5, alpha=0.6), start=1):
    print(f"- ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i]}")
    rows_hybrid.append({
        "rank": k,
        "score": float(s),
        "doc_id": int(i),
        "label": label_names[corpus_labels[i]],
        "text": corpus_texts[i]
    })

- (0.298) [World] Election-Year Rate Hike Puzzles Some WASHINGTON - Going against conventional wisdom, the Federal Reserve is raising interest rates in an election year. And it is Fed Chairman Alan Greenspan, a Republican, who is leading the charge even though an incumbent Republican in the White House is facing voter unrest about the state of the economy...
- (0.282) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
- (0.279) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market this week during the depth of the\summer doldrums.
- (0.270) [Business] Oil and Economy Cloud Stocks' Outlook  NEW YORK (Reuters) - Soaring crude prices plus worries  about the econo

In [17]:
# ------------------------
# CSV 저장 (각각)
# ------------------------
# - 출력 경로: ./outputs/
# - 인코딩: UTF-8 (엑셀에서 바로 열 때는 'utf-8-sig'도 고려)
out = os.path.join(os.path.abspath("."), "outputs")
os.makedirs(out, exist_ok=True)

pd.DataFrame(rows_tfidf).to_csv(os.path.join(out, "tfidf_top5.csv"),
                                index=False, encoding="utf-8")
pd.DataFrame(rows_sbert).to_csv(os.path.join(out, "sbert_top5.csv"),
                                index=False, encoding="utf-8")
pd.DataFrame(rows_hybrid).to_csv(os.path.join(out, "hybrid_top5.csv"),
                                 index=False, encoding="utf-8")



### 4. 결과 비교: 직관적 성능 분석(토픽/키워드/문맥)


In [18]:
# ------------------------
# 비교용 테이블/프린트
# ------------------------

def show_results(title: str, pairs):
    """
    검색 결과 목록을 깔끔하게 프린트하는 유틸
    Args:
        title (str): 섹션 제목
        pairs (List[Tuple[int, float]]): (문서 인덱스, 점수) 리스트
    """
    print(f"\n[{title}]")
    for rank, (i, s) in enumerate(pairs, start=1):
        # 줄바꿈 제거 + 앞부분만 미리보기
        txt = corpus_texts[i].replace("\n", " ")
        print(f"{rank:>2}. ({s:.3f}) [{label_names[corpus_labels[i]]}] {txt}")

# 각 검색기에서 상위 5개 문서 인덱스/점수 가져오기
r_tfidf = search_tfidf(query, topk=5)
r_sbert = search_sbert(query, topk=5)
r_hybrid = search_hybrid(query, topk=5, alpha=0.6)  # SBERT 가중치 0.6

# 결과 프린트
show_results("TF-IDF Top-5", r_tfidf)
show_results("SBERT Top-5", r_sbert)
show_results("Hybrid(0.6) Top-5", r_hybrid)



[TF-IDF Top-5]
 1. (0.188) [Business] Veteran inventor in market float Trevor Baylis, the veteran inventor famous for creating the Freeplay clockwork radio, is planning to float his company on the stock market.
 2. (0.180) [Business] In a Down Market, Head Toward Value Funds There is little cause for celebration in the stock market these days, but investors in value-focused mutual funds have reason to feel a bit smug -- if only because they've lost less than the folks who stuck with growth.
 3. (0.172) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
 4. (0.172) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market this week during the depth of the\summer d

In [19]:

# ------------------------
# 간단한 직관 비교: 레이블 빈도
# ------------------------
from collections import Counter

def label_counts(pairs):
    """
    (문서 인덱스, 점수) 리스트 → 레이블명 카운트 딕셔너리
    """
    return Counter([label_names[corpus_labels[i]] for i, _ in pairs])

print("\n[레이블 빈도 비교]")
for name, pairs in [("TF-IDF", r_tfidf), ("SBERT", r_sbert), ("Hybrid", r_hybrid)]:
    print(name, dict(label_counts(pairs)))



[레이블 빈도 비교]
TF-IDF {'Business': 5}
SBERT {'World': 2, 'Business': 3}
Hybrid {'World': 1, 'Business': 4}


In [20]:

# ------------------------
# CSV 저장 (r_* 바로 직렬화)
# ------------------------
# - rank/score/doc_id/label/text 컬럼으로 저장
# - 엑셀 호환이 필요하면 encoding="utf-8-sig" 권장
rows_tfidf  = [{"rank": k, "score": float(s), "doc_id": int(i),
                "label": label_names[corpus_labels[i]], "text": corpus_texts[i]}
               for k, (i, s) in enumerate(r_tfidf, start=1)]
rows_sbert  = [{"rank": k, "score": float(s), "doc_id": int(i),
                "label": label_names[corpus_labels[i]], "text": corpus_texts[i]}
               for k, (i, s) in enumerate(r_sbert, start=1)]
rows_hybrid = [{"rank": k, "score": float(s), "doc_id": int(i),
                "label": label_names[corpus_labels[i]], "text": corpus_texts[i]}
               for k, (i, s) in enumerate(r_hybrid, start=1)]

out = os.path.join(os.path.abspath("."), "outputs")
os.makedirs(out, exist_ok=True)

pd.DataFrame(rows_tfidf ).to_csv(os.path.join(out, "tfidf_top5_cell14.csv" ), index=False, encoding="utf-8")
pd.DataFrame(rows_sbert ).to_csv(os.path.join(out, "sbert_top5_cell14.csv" ), index=False, encoding="utf-8")
pd.DataFrame(rows_hybrid).to_csv(os.path.join(out, "hybrid_top5_cell14.csv"), index=False, encoding="utf-8")




### 5. 추가: 하이브리드 가중치/속도/stopwords 영향


In [21]:
# ------------------------
# 하이브리드 가중치 변화 예시
# ------------------------
# - search_hybrid: alpha ∈ [0,1]에서 SBERT vs TF-IDF 비중을 조절
#   alpha=1.0 → SBERT 100%, alpha=0.0 → TF-IDF 100%
# - 상위 5개 문서의 라벨 분포가 alpha 변화에 따라 어떻게 달라지는지 직관적으로 확인
for a in [0.0, 0.3, 0.6, 0.9, 1.0]:
    pairs = search_hybrid(query, topk=5, alpha=a)
    top_labels = [label_names[corpus_labels[i]] for i, _ in pairs]
    print(f"alpha={a:.1f} -> {top_labels}")

# ------------------------
# 속도 비교
# ------------------------
# - timer: 사용자 정의 컨텍스트 매니저(벽시계 시간 측정)라고 가정
# - 유의: SBERT는 최초 호출 시 모델 로드/웨이트 준비로 인해 첫 타이밍이 더 느릴 수 있음(콜드 스타트).
#         공정한 비교를 원하면 워밍업 쿼리를 1~2회 먼저 실행하세요.
with timer("TF-IDF 질의 시간"):
    _ = search_tfidf(query, topk=10)

with timer("SBERT 질의 시간"):
    _ = search_sbert(query, topk=10)

with timer("Hybrid(alpha=0.6) 질의 시간"):
    _ = search_hybrid(query, topk=10, alpha=0.6)

alpha=0.0 -> ['Business', 'Business', 'Business', 'Business', 'Business']
alpha=0.3 -> ['Business', 'Business', 'Business', 'Business', 'Business']
alpha=0.6 -> ['World', 'Business', 'Business', 'Business', 'Business']
alpha=0.9 -> ['World', 'Business', 'Business', 'Business', 'Business']
alpha=1.0 -> ['World', 'Business', 'Business', 'Business', 'World']
[TIME] TF-IDF 질의 시간: 0.00s
[TIME] SBERT 질의 시간: 2.33s
[TIME] Hybrid(alpha=0.6) 질의 시간: 2.26s


In [22]:
# ------------------------
# stopwords 영향: 불용어 제거 on/off 비교
# ------------------------
# - 기존 vectorizer는 stop_words=stop_words로 설정되어 있다고 가정
# - 아래는 불용어 제거를 하지 않는 대안 벡터라이저를 만들어 성능 차이를 관찰
# - 주의: ngram_range=(1,2), max_features=30000은 메모리/시간 트레이드오프가 큼
alt_vectorizer = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    stop_words=None   # 불용어 제거 없음
)

with timer("TF-IDF(no stopwords) fit_transform"):
    # 같은 코퍼스에 대해 새로운 어휘 사전과 TF-IDF 행렬을 학습/생성
    X_tfidf_raw = alt_vectorizer.fit_transform(corpus_texts)

def search_tfidf_raw(q: str, topk: int = 5):
    """
    불용어 제거를 하지 않은 대안 벡터라이저(alt_vectorizer)로 검색.
    Returns: [(문서 인덱스, 코사인 유사도), ...]
    """
    qv = alt_vectorizer.transform([q])
    sims = cosine_similarity(qv, X_tfidf_raw)[0]
    idx = np.argsort(-sims)[:topk]
    return [(int(i), float(sims[i])) for i in idx]

[TIME] TF-IDF(no stopwords) fit_transform: 0.15s


In [23]:
# ------------------------
# 결과 비교 출력
# ------------------------
# - 같은 질의에 대해 stopwords 사용/미사용의 상위 문서가 어떻게 달라지는지 비교
print("\n[TF-IDF(stopwords) vs TF-IDF(raw) 비교]")
print("- with stopwords:")
for i, s in search_tfidf(query, topk=3):
    print(f"  ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i][:100]}...")

print("- no stopwords:")
for i, s in search_tfidf_raw(query, topk=3):
    print(f"  ({s:.3f}) [{label_names[corpus_labels[i]]}] {corpus_texts[i][:100]}...")

# 참고:
# - 속도 비교는 단일 실행(run) 결과이므로 변동성 있음 → timeit/반복 평균을 쓰면 더 신뢰도 높음.
# - Hybrid 점수는 스케일 차이의 영향을 받을 수 있음 → 필요 시 각 점수를 MinMax/Z-score로 정규화 후 합산.
# - 공정 비교를 위해 전처리(토큰화/소문자화/stemming 등)는 동일하게 유지하는 것이 좋음.
# - 매우 긴 문서/질의가 섞이면 TF-IDF는 길이 보정(sublinear_tf=True 등), SBERT는 문장 길이 분포에 주의.




[TF-IDF(stopwords) vs TF-IDF(raw) 비교]
- with stopwords:
  (0.188) [Business] Veteran inventor in market float Trevor Baylis, the veteran inventor famous for creating the Freepla...
  (0.180) [Business] In a Down Market, Head Toward Value Funds There is little cause for celebration in the stock market ...
  (0.172) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about th...
- no stopwords:
  (0.179) [Business] Hungarian central bank cuts key interest rate by half percentage point (AFP) AFP - The Hungarian cen...
  (0.157) [Business] Veteran inventor in market float Trevor Baylis, the veteran inventor famous for creating the Freepla...
  (0.132) [Business] Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about th...
